In [1]:
import sys
import nltk
import sklearn
import pandas
import numpy

print('python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Sklearn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('numpy: {}'.format(numpy.__version__))





python: 3.8.8 (default, Apr 13 2021, 12:59:45) 
[Clang 10.0.0 ]
NLTK: 3.6.1
Sklearn: 0.24.1
Pandas: 1.2.4
numpy: 1.19.5


## Loading Dataset

In [5]:
import pandas as pd
import numpy as np

# loading SMS dataset 
data = pd.read_table('SMSSpamCollection', header = None, encoding ='utf-8' )

In [4]:
#gathering information about loaded dataset
print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5572 non-null   object
 1   1       5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None
      0                                                  1
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [6]:
# Checking for class distribution
classes = data[0]
print(classes.value_counts())

ham     4825
spam     747
Name: 0, dtype: int64


## Pre processing the data

In [7]:
# converting class labels into binary values
# ham = 0, spam = 1
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform(classes)
print(classes[:10])
print(Y[:10])

0     ham
1     ham
2    spam
3     ham
4     ham
5    spam
6     ham
7     ham
8    spam
9    spam
Name: 0, dtype: object
[0 0 1 0 0 1 0 0 1 1]


In [8]:
#storing the 
text_messages = data[1]
print(text_messages[:10])

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
5    FreeMsg Hey there darling it's been 3 week's n...
6    Even my brother is not like to speak with me. ...
7    As per your request 'Melle Melle (Oru Minnamin...
8    WINNER!! As a valued network customer you have...
9    Had your mobile 11 months or more? U R entitle...
Name: 1, dtype: object


In [10]:
# Using regular expressions for simplyfying the data
processed = text_messages.str.replace('^.+@[^\.].*\.[a-z]{2,}$', 'emailaddress')
processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$',
                                  'webaddress')
processed = processed.str.replace(r'£|\$', 'moneysymb')
processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$','phonenumbr')
# Replace numbers with 'numbr'
processed = processed.str.replace(r'\d+(\.\d+)?', 'numbr')
# Remove punctuation
processed = processed.str.replace(r'[^\w\d\s]', ' ')
# Replace whitespace between terms with a single space
processed = processed.str.replace(r'\s+', ' ')
# Remove leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '')
# changing words to lower case 
processed = processed.str.lower()
print(processed)

0       go until jurong point crazy available only in ...
1                                 ok lar joking wif u oni
2       free entry in numbr a wkly comp to win fa cup ...
3             u dun say so early hor u c already then say
4       nah i don t think he goes to usf he lives arou...
                              ...                        
5567    this is the numbrnd time we have tried numbr c...
5568                  will ü b going to esplanade fr home
5569    pity was in mood for that so any other suggest...
5570    the guy did some bitching but i acted like i d...
5571                            rofl its true to its name
Name: 1, Length: 5572, dtype: object


<ipython-input-10-663d72559e4c>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = text_messages.str.replace('^.+@[^\.].*\.[a-z]{2,}$', 'emailaddress')
<ipython-input-10-663d72559e4c>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$',
<ipython-input-10-663d72559e4c>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = processed.str.replace(r'£|\$', 'moneysymb')
<ipython-input-10-663d72559e4c>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$','phonenumbr')
<ipython-input-10-663d72559e4c>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = processed.str.re

In [13]:
from nltk.corpus import stopwords

# remove stop words from text messages

stop_words = set(stopwords.words('english'))

processed = processed.apply(lambda x: ' '.join(
    term for term in x.split() if term not in stop_words))

In [12]:
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sumedhreddy/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
# Remove word stems using a Porter stemmer
ps = nltk.PorterStemmer()

processed = processed.apply(lambda x: ' '.join(
    ps.stem(term) for term in x.split()))
